# Functions and Settings

This notebook contains various functions and settings used by the notebooks for the Data Reduction Workshop.

- [Imports and basic setup](#setup)
- [Progress Bar](#progress)
- [Load a file](#load)
- [Plots](#plots)
- [Gas Calibration](#gascal)

### Imports and basic setup
<a id='setup'></a>

In [ ]:
# General data imports
import datetime
import pandas
import numpy as np
import math
from scipy import stats

# Jupyter imports
from ipywidgets import interact, widgets

# Plotting imports
from bokeh.io import output_notebook, push_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Select, CustomJS
from bokeh.layouts import column, row
output_notebook()

import folium

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Other options
pandas.options.display.max_columns = None
pandas.options.display.max_rows = None


### Progress Bar
<a id="load"></a>
Display a progress bar for long actions

From https://github.com/kuk/log-progress

In [ ]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

### Load a file
<a id="load"></a>
All files used in this workshop use the same format. This function loads a requested file and into a pandas dataframe ensures that all the data types are set correctly.

In [ ]:
def load_file(filename):
  # Load the file
  df = pandas.read_csv(filename, sep="\t")
  
  # Set the Timestamp column as datetime
  df['Timestamp'] = df['Timestamp'].apply(pandas.to_datetime)

  return df

### Plots
<a id="plots"></a>
Some shortcut functions to make plots

In [ ]:
def scatter_plot(xdata, ydata, xlabel, ylabel):
  p = figure(plot_width=600, plot_height=600, x_axis_label=xlabel, y_axis_label=ylabel)
  p.circle(xdata, ydata, size=5)
  return p

def time_series(xdata, ydata, ylabel):
  p = figure(plot_width=600, plot_height=600, x_axis_type='datetime', x_axis_label='Time', y_axis_label=ylabel)
  p.circle(xdata, ydata, size=5)
  return p


### Gas Calibration
<a id="gascal"></a>
Regression functions for gas calibration notebook

In [ ]:
### Functions for Gas Calibration: (8)

def gascal_calc_regression(standard_indices):
    measured = []
    ref = []
    
    converted_slope = None
    converted_intercept = None
    
    if len(standard_indices) > 1:
        for i in standard_indices:
            measured.append(standard_measurements['CO2 um/m'][i])
            ref.append(standard_measurements['Reference'][i])

        slope, intercept, r, p, std_err = stats.linregress(measured, ref)

        # Convert slope and intercept
        converted_slope = 1 / slope
        converted_intercept = (intercept * -1) / slope

    return converted_slope, converted_intercept

def gascal_plot_regression8(plot, standard_indices, name, color):
    measured = []
    ref = []

    existing_line = plot.select_one({'name': name})
    if existing_line is not None:
        plot.renderers.remove(existing_line)

    if (len(standard_indices) > 1):
        converted_slope, converted_intercept = gascal_calc_regression(standard_indices)
        plot.line([0, 550],
                  [0 * converted_slope + converted_intercept, 550 * converted_slope + converted_intercept],
                  line_dash='dashed', line_color=color, name=name)
    
    
    push_notebook()

def gascal_get_standard_indices(STD1, STD2, STD3, STD4):
    indices = []
    if STD1:
        indices.append(s1 + 0)
    if STD2:
        indices.append(s1 + 1)
    if STD3:
        indices.append(s1 + 2)
    if STD4:
        indices.append(s1 + 3)
        
    return indices

def gascal_standard_chosen8(STD1, STD2, STD3, STD4):
    chosen_indices = gascal_get_standard_indices(STD1, STD2, STD3, STD4)
    gascal_plot_regression8(plot, chosen_indices, 'chosen', 'red')
